# 유튜브 추천 알고리즘 설계
1. 고객 데이터를 문장 형식으로 추출했습니다. 유튜브 데이터가 모두 텍스트 형식으로 되어있기에 동일한 pre-trained 모델을 활용해 임베딩함으로써 같은 벡터 공간을 공유하게 됩니다.  
2. 유튜브 데이터 중 'title'과 'content'를 LLM에 집어넣어 정제합니다. 정제된 하나의 텍스트는 임베딩에 활용됩니다.  
3. 텍스트 형식인 고객 데이터(1개)와 유튜브 데이터(다수)를 임베딩합니다.  
4. 임베딩된 고객 데이터(백터)와 유튜브 데이터(행렬) 간 코사인 유사도를 측정합니다.
5. 유사도를 기준으로 내림차순 정렬하여 고객의 정보와 가장 유사한 유튜브의 제목과 url을 제공합니다.

### 1. 고객 데이터 생성

In [2]:
import pandas as pd
import time
from ClovaX_Executor import CompletionExecutor
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# 특정 고객의 데이터 생성
customer_data = {
    'CustomerID': [1],
    'Age': [65],
    'Gender': ['남자'],
    'Location': ['서울'],
    'Occupation': ['은퇴'],
    'Financial_Literacy_Level': ['보통'],
    'Investment_Horizon': ['장기투자'],
    'Risk_Tolerance': ['보수적'],
    'Preferred_Investment_Types': ['주식, 채권'],
    'Investment_Amount': [1000000],  # 투자 금액 (단위: 원)
    'Past_Investment_Experience': ['보통'],
    'Preferred_Investment_Topics': ['2차전지 관련주', '여행 관련주'],
    'Savings_Account': [0.65],
    'Bonds': [0.15],
    'Stocks': [0.2]
}

# 고객 특징 설명 글 생성
def generate_customer_description(customer_data):
    description = (
        f"이 고객은 {customer_data['Location'][0]}에 거주하는 {customer_data['Age'][0]}세의 {customer_data['Gender'][0]}입니다. "
        f"현재 직업은 {customer_data['Occupation'][0]}이며, 금융 이해도는 {customer_data['Financial_Literacy_Level'][0]} 수준입니다. "
        f"투자 기간은 {customer_data['Investment_Horizon'][0]}으로, 위험 감수 성향은 {customer_data['Risk_Tolerance'][0]}입니다. "
        f"주로 선호하는 투자 유형은 {customer_data['Preferred_Investment_Types'][0]}이며, 현재 투자 가능한 금액은 {customer_data['Investment_Amount'][0]}만원입니다. "
        f"과거 투자 경험은 {customer_data['Past_Investment_Experience'][0]} 수준이며, 선호하는 투자 주제는 {customer_data['Preferred_Investment_Topics'][0]}입니다. "
        f"현재 자산 구성 비율은 저축 계좌에 {customer_data['Savings_Account'][0]*100}%, 채권에 {customer_data['Bonds'][0]*100}%, 주식에 {customer_data['Stocks'][0]*100}%로 분포되어 있습니다."
    )
    return description

# 고객 특징 설명 글 출력
customer_feature_text = generate_customer_description(customer_data)
customer_feature_text

c:\Users\cksdu\anaconda3\envs\miraeasset\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


'이 고객은 서울에 거주하는 65세의 남자입니다. 현재 직업은 은퇴이며, 금융 이해도는 보통 수준입니다. 투자 기간은 장기투자으로, 위험 감수 성향은 보수적입니다. 주로 선호하는 투자 유형은 주식, 채권이며, 현재 투자 가능한 금액은 1000000만원입니다. 과거 투자 경험은 보통 수준이며, 선호하는 투자 주제는 2차전지 관련주입니다. 현재 자산 구성 비율은 저축 계좌에 65.0%, 채권에 15.0%, 주식에 20.0%로 분포되어 있습니다.'

### 2. 유튜브 데이터 정제

In [10]:
pd.set_option('display.max_colwidth', None)

In [86]:
youtube = pd.read_csv('youtube_data.csv')[['title','content','video_url']]
youtube['content'] = youtube['content'].apply(lambda x : x[:512])
youtube.head(8)

,title,content,video_url
0,"[슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이? #중국증시 #로보락",【이인구의 슬기로운 투자 생각】 ep4. 중국투자전략\n출연: 이인구 이사(미래에셋증권 디지털리서치팀)\n\n00:00 하이라이트\n00:25 중국 매크로 점검 \n03:35 중국 해외 무역 비중 변화\n05:21 중국 주요 가전업체 주가 \n06:20 클로징\n\n#중국증시 #투자전략 #이인구 #미래에셋증권 #로보락 #roborock \n#KOR #ENG #IND \n\n[▼보고서 보기▼]\n[중국 투자 전략] 경기 회복의 새로운 시그널\nhttps://securities.miraeasset.com/bbs...\n챕터\n모두 보기\n하이라이트\n0:00\n중국 매크로 점검\n0:25\n중국 해외 무역 비중 변화\n3:35\n중국 주요 가전업체 주가\n5:21\n스크립트\n스크립트를 보면서 시청하세요.\n스크립트 표시\n미래에셋 스마트머니\n구독자 149만명\n동영상\n정보\n페이스북\n[슬기로운 투자 생각] 지금이 진짜 중국 증시 저점일까?\n제작자: 미래에셋 스마트머니\n조회수 2.1만회 · 3개월 전\n간략히,https://www.youtube.com/watch?v=JrHgv0R8RIQ&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=1
1,"[슬기로운 투자 생각] 중국 고배당주 ETF, 투자 열풍 이유는? #Hangseng #중국주식 #고배당주","【이인구의 슬기로운 투자 생각】\nep5. 중국 Go! 배당 주!\n\n00:00 하이라이트\n00:49 중국 배당주 투자 열풍\n01:37 투자대안의 부재와 높은 배당수익률\n03:50 중국판 밸류업! 신국9조 \n05:08 홍콩배당주는 세제 혜택?\n06:35 배당+00은 더욱 중요!\n09:10 중국, 홍콩 증시의 새로운 투자 물결..고배당 투자\n09:50 클로징\n\n[슬기로운 투자 생각] ▼ 더보기\nep.1. 지금이 진짜 중국 증시 저점일까?\n • [슬기로운 투자 생각] 지금이 진짜 중국 증시 저점일까? \n\nep.2. 중국 증시 저점? 중국의 새로운 주도주는?\n • [슬기로운 투자 생각] 중국 증시 저점? 중국의 새로운 주도주는? \n\nep.3. 중국의 전략적 투자 방향?\n • [슬기로운 투자 생각] 중국의 전략적 투자 방향? \n\nep.4.중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이?\n • [슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는... \n\n\n[▼보",https://www.youtube.com/watch?v=u1j5cB9Dqpc&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=2
2,"반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해"" #CATL | 슬기로운 투자 생각","【이인구의 슬기로운 투자 생각】\nep.6 반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해""\n\n00:00 하이라이트\n00:31 주가 하락, 나는 진짜 억울!\n01:31 반등 시동! 글로벌 1위 배터리 업체\n02:18 한중 주요 배터리제조업체 수익성 비교\n02:53 CATL 주가 및 PER 밴드 \n03:31 클로징\n\n출연: 이인구 이사(미래에셋증권 디지털리서치팀)\n\n[슬기로운 투자 생각] ▼ 더보기\nep.1. 지금이 진짜 중국 증시 저점일까?\n • [슬기로운 투자 생각] 지금이 진짜 중국 증시 저점일까? \n\nep.2. 중국 증시 저점? 중국의 새로운 주도주는?\n • [슬기로운 투자 생각] 중국 증시 저점? 중국의 새로운 주도주는? \n\nep.3. 중국의 전략적 투자 방향?\n • [슬기로운 투자 생각] 중국의 전략적 투자 방향? \n\nep.4. 중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이?\n • [슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는...",https://www.youtube.com/watch?v=Ms6p1HznC_U&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=3
3,"🌝중국 주도주 반등 랠리, 제대로 시동겁니다. 🚌 놓치기 전에 타세요~ [슬기로운 투자 생각] ep.7 #중국주식 #주도주","【이인구의 슬기로운 투자 생각】\nep.6 반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해""\n\n00:00 하이라이트\n00:31 주가 하락, 나는 진짜 억울!\n01:31 반등 시동! 글로벌 1위 배터리 업체\n02:18 한중 주요 배터리제조업체 수익성 비교\n02:53 CATL 주가 및 PER 밴드 \n03:31 클로징\n\n출연: 이인구 이사(미래에셋증권 디지털리서치팀)\n\n[슬기로운 투자 생각] ▼ 더보기\nep.1. 지금이 진짜 중국 증시 저점일까?\n • [슬기로운 투자 생각] 지금이 진짜 중국 증시 저점일까? \n\nep.2. 중국 증시 저점? 중국의 새로운 주도주는?\n • [슬기로운 투자 생각] 중국 증시 저점? 중국의 새로운 주도주는? \n\nep.3. 중국의 전략적 투자 방향?\n • [슬기로운 투자 생각] 중국의 전략적 투자 방향? \n\nep.4. 중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이?\n • [슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는...",https://www.youtube.com/watch?v=7Yoej1DUR_0&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=4
4,[유토피아] AI는 Apple iphone의 줄임말? | 2024.5.17(금),"#apple #ai #openai\n\n✨ 쏟아지는 AI 관련 홍수 속에서 꼭 알아야 할 Highlight를 엄선해드립니다.\n✨ 이 콘텐츠 제작에 ChatGPT, Opus 등 각종 AI들이 활용되었습니다.\n\n00:00 시작\n00:08 진짜 시작\n00:33 Open AI의\n Spring Update\n04:40 GPT-4o의 성능\n06:00 AI Peer Table\n\n\n[▼관련 리포트 보기▼]\n\nAI Weekly #15\nhttps://securities.miraeasset.com/bbs...\n\nAI Weekly #16\nhttps://securities.miraeasset.com/bbs...\n\n\n스마트머니 구독 : https://bit.ly/3lluWtU\n챕터\n모두 보기\n시작\n0:00\n진짜 시작\n0:08\nOpen AI의 Spring Update\n0:33\nGPT-4o의 성능\n4:40\n스크립트\n스크립트를 보면서 시청하세요.\n스크립트 표시\n미래에셋 스마트머니\n구독자 149만명\n동영상\n정보\n페이스북\n간략히",https://www.youtube.com/watch?v=GDHMzjCLSjk&list=PL1-YXgQy7nWx-kLXcBKDmf8IdGAdyAbpu&index=1
5,[유토피아] 중국에서 먼저 완전자율주행 출시하는 테슬라? | 2024.5.3(금),"#테슬라 #마크저커버그 #ai #llama3 \n\n✨ 쏟아지는 AI 관련 홍수 속에서 꼭 알아야 할 Highlight를 엄선해드립니다.\n✨ 이 콘텐츠 제작에 ChatGPT, Opus 등 각종 AI들이 활용되었습니다.\n\n00:00 시작\n00:08 마크 저커버그의 Llama3\n03:14 눈에 띄는 중화권 기업들\n05:45 AI에서 빠지지 않는 테슬라\n09:30 AI Peer Table\n\n\n[▼관련 리포트 보기▼]\n\nAI Weekly #13\nhttps://securities.miraeasset.com/bbs...\n\nAI Weekly #14\nhttps://securities.miraeasset.com/bbs...\n\n스마트머니 구독 : https://bit.ly/3lluWtU\n챕터\n모두 보기\n시작\n0:00\n마크 저커버그의 Llama3\n0:08\n눈에 띄는 중화권 기업들\n3:14\nAI에서 빠지지 않는 테슬라\n5:45\n스크립트\n스크립트를 보

In [37]:
summary = []

for i in range(len(youtube)):
    completion_executor = CompletionExecutor(
    host='https://clovastudio.stream.ntruss.com',
    api_key='NTA0MjU2MWZlZTcxNDJiY63IHRsuPbkmleZBGwzFCDyaHMUZtTU7D+D0PfCHfrjO',
    api_key_primary_val='UdrE1xG6afhw1J7RnoYIjg8SHNGeBOc4pzquDSLn',
    request_id='d4f031d7-c91c-4176-9e7a-e53e350e4f7b'
    )

    preset_text = [
        {"role":"system","content":"<명령문>\r\n주어지는 title과 information는 유튜브 영상의 제목과 정보입니다. 이를 바탕으로 유튜브 영상의 주요 포인트를 요약해서 output으로 제공해주세요. 특히 information에서 영상의 내용과 관련있는 정보만 따로 추출해서 가져와야 합니다. 이 요약은 다음의 기준을 따릅니다\r\n\r\n\r\n- 제목이 핵심입니다. 제목을 바탕으로 내용을 유추해보세요.\r\n- 반드시 줄글로 전달하세요. 개행문자는 사용하지 않습니다.\r\n- 제목과 관련한 내용으로 작성을 시작하고, 시간대에 따른 내용을 작성해주세요. 마무리는 한 줄 요약을 해주며 시청자에게 유용한 정보를 제공해주세요.\r\n- 제시된 내용 이외의 정보는 제공하지 마세요.\r\n\r\n\n아래 예시를 참고하세요. 그리고 주어지는 title(제목)과 information(정보)를 바탕으로 output(요약)을 도출하세요.\n\n\n출력문은 output(요약)만으로 구성됩니다.\r\n\n\n<예시1>\r\n### title : '\''[유토피아] AI는 Apple iphone의 줄임말? | 2024.5.17(금)'\''\r\n### information : \"\"\"#apple #ai #openai\\n\\n✨ 쏟아지는 AI 관련 홍수 속에서 꼭 알아야 할 Highlight를 엄선해드립니다.\\n✨ 이 콘텐츠 제작에 ChatGPT, Opus 등 각종 AI들이 활용되었습니다.\\n\\n00:00 시작\\n00:08 진짜 시작\\n00:33 Open AI의\\n Spring Update\\n04:40 GPT-4o의 성능\\n06:00 AI Peer Table\\n\\n\\n[▼관련 리포트 보기▼]\\n\\nAI Weekly #15\\nhttps://securities.miraeasset.com/bbs...\\n\\nAI Weekly #16\\nhttps://securities.miraeasset.com/bbs...\\n\\n\\n스마트머니 구독 : https://bit.ly/3lluWtU\\n챕터\\n모두 보기\\n시작\\n0:00\\n진짜 시작\\n0:08\\nOpen AI의 Spring Update\\n0:33\\nGPT-4o의 성능\\n4:40\\n스크립트\\n스크립트를 보면서 시청하세요.\\n스크립트 표시\\n미래에셋 스마트머니\\n구독자 149만명\\n동영상\\n정보\\n페이스북\\n간략히\"\"\"\r\n\r\n### Chain-of-Thought\r\ntitle을 보면 AI와 apple의 관계성에 대해 의문을 가지고 있어. information 시작 부분을 통해 해당 영상은 AI 홍수 속에서 꼭 알아야 하는 정보를 제공한다는 것을 이해한 후 들어가보자. 그 뒤에 00:00 시작부터 각 시점마다 어떤 내용을 이야기하고 있는지 설명하고 있어. 영상의 주 내용인 Open AI와 GPT-4o에 대한 간단한 설명을 덧붙이자. \r\n그 뒤의 내용은 영상에서 전달하고자 하는 내용과 관련이 없기 때문에 과감하게 생략해도 돼. AI의 최신 동향에 관심이 있는 경우 해당 영상이 유용할 수 있다고 설명하며 마무리 하는게 좋을 것 같아.\r\n\r\n### output \r\n이 영상에서는 \"AI는 Apple iPhone의 줄임말?\"이라는 제목을 통해 AI와 Apple의 관계를 흥미롭게 탐구합니다. 0:08에서 본격적으로 시작하며, 0:33에는 Open AI의 Spring Update에 대해 다룹니다. 이어서 4:40에는 최신 인공지능 모델 GPT-4o의 성능을 분석하고, 6:00에는 AI Peer Table에 대한 내용을 소개합니다. Open AI는 인공지능 연구소로 GPT-4o는 더욱 발전된 자연어 처리 능력을 자랑합니다. 이 영상은 AI 관련 주요 업데이트와 기술의 발전을 이해하는 데 도움을 줍니다. 결론적으로, AI의 최신 동향을 파악하고 싶다면 이 영상을 시청하는 것이 유용합니다.\r\n\r\n\r\n<예시2>\r\n# title : '\''[Charting Disruption] 디지털헬스케어, AI& 수술로봇 (#SUB #ENG #KOR)'\''\r\n\r\n# information : \"\"\"글로벌 ETF 운용사 Global X의 Charting Disruption 시리즈!\\n\\n[Digitizing Medicine]\\n출연: Arelis Agosto Global X research analyst\\n\\n\\n[AI] 생성형 AI의 혁신적 파워\\n • [Charting Disruption] 생성형 AI의 혁신적 파워,... \\n\\n[디지털헬스케어] 디지털헬스케어, AI& 수술로봇\\n • [Charting Disruption] 디지털헬스케어, AI& 수술... \\n\\n[로보틱스] 스마트 머신의 부상\\n • [Charting Disruption] 스마트 머신의 부상 #Ro... \\n\\n[전기 미래] 탈탄소화를 위한 전기차 배터리 기술\\n • 동영상 \\n\\n#ChartingDisruption #MiraeAsset #GlobalX #ETF\\n🌎 Enable subtitles for ENG/KOR/JAP/ZH-CHT/ESP/IDN lyrics!\\n 영어/일본어/한국어/중국간체/스페인어/인도네시아어/힌디어(번역은 AI솔루션 사용)\\n🙏'\''설정'\''에서 자막-원하는 자막을 선택하시고 '\''CC'\''를 눌러보세요. \\n🔊스마트머니 구독 : https://bit.ly/3lluWtU\\n스크립트\\n스크립트를 보면서 시청하세요.\\n스크립트 표시\\n미래에셋 스마트머니\\n구독자 149만명\\n동영상\\n정보\\n페이스북\\n간략히\"\"\"\r\n\r\n### Chain-of-Thought\r\n\r\ntitle을 보면 디지털 헬스케어와 AI, 수술로봇에 관련한 영상임을 유추할 수 있어. 이후 information 시작 부분에서 \"글로벌 ETF 운용사 Global X의 Charting Disruption 시리즈!\"를 통해 이 영상이 Global X의 시리즈의 일부임을 알 수 있어. \"출연: Arelis Agosto Global X research analyst\" 부분에서 Arelis Agosto가 출연하며 내용을 전달할 것임을 알 수 있어. [AI], [Charting Disruption], [디지털헬스케어], [Charting Disruption], [로보틱스], [전기 미래]에 대한 내용을 다루는 것을 확인할 수 있으니까 이에 대한 내용도 넣으면 좋을 것 같아.\r\n그 뒤의 내용은 영상에서 전달하고자 하는 내용과 관련이 없기 때문에 과감하게 생략해도 돼. 최신 디지털 헬스케어에 관심 있는 사람들을 위한 마무리 멘트로 요약을 끝맺자.\r\n\r\n\r\n### output\r\n이 영상에서는 \"디지털헬스케어, AI& 수술로봇\"이라는 제목을 통해 디지털 헬스케어와 관련된 최신 기술들을 탐구합니다. Global X의 Charting Disruption 시리즈로, 연구 애널리스트 Arelis Agosto가 출연하여 생성형 AI의 혁신적 파워, 디지털 헬스케어, AI와 수술 로봇, 스마트 머신의 부상, 전기차 배터리 기술과 탈탄소화에 대해 다룹니다. 이 영상은 디지털 헬스케어와 관련된 주요 기술 및 트렌드를 이해하는 데 매우 유용합니다. 최신 디지털 헬스케어 트렌드를 파악하고 싶다면 이 영상을 시청하는 것이 좋습니다."},
        {"role": "user", "content": f"다음 유튜브의 정보를 파악하고 요약하세요. 요약문은 반드시 줄글로 작성해야 합니다.  \n\n### title : {youtube.iloc[i].title}\n### information : {youtube.iloc[i].content}\n\n output :"}
    ]

    request_data = {
        'messages': preset_text,
        'topP': 0.8,
        'topK': 0,
        'maxTokens': 512,
        'temperature': 0.1,
        'repeatPenalty': 1.2,
        'stopBefore': [],
        'includeAiFilters': True,
        'seed': 0
    }
    completion_executor.execute(request_data)
    summary.append(completion_executor.full_response)
    print(completion_executor.full_response)

    if i % 3 == 0:
        time.sleep(20)

이 영상에서는 "소비자 심리와 생산자 물가"라는 제목을 통해 미국 증시의 주요 체크포인트를 다룹니다. 0:14에서는 소비자 심리를, 2:24에서는 생산자 물가를 분석합니다. 이 영상은 미국 증시의 흐름을 파악하고 투자 결정을 내리는 데 도움이 되는 정보를 제공합니다. 미국 증시의 소비자 심리와 생산자 물가에 대한 정보를 얻고 싶다면 이 영상을 시청하는 것이 좋습니다.
이 영상에서는 "7월 CPI 결과는?"이라는 제목을 통해 미국 증시의 주요 체크포인트 중 하나인 물가에 대해 다룹니다. 7월 CPI 결과 발표를 앞두고, 슈퍼코어 CPI 등 물가 관련 지표들을 살펴봅니다. 또한, 주식 시장 리뷰와 미 증시 전망, 글로벌 주식 시장 변화와 전망에 대한 디지털 리서치 보고서도 제공합니다. 미국 증시와 물가에 대한 정보를 얻고 싶다면 이 영상을 시청하는 것이 유용합니다.
이 영상에서는 "미국증시 체크포인트"라는 제목을 통해 미국 증시와 관련된 주요 이슈들을 다룹니다. 0:14에서는 주간 모기지 신청건수를 다루고, 1:21에서는 국제유가에 대해 살펴봅니다. 미국 증시와 관련된 다양한 지표와 이슈들을 이해하는 데 도움이 되는 정보를 제공합니다. 미국 증시의 동향을 파악하고자 하는 투자자들에게 유용한 자료입니다.
이 영상에서는 "뉴욕 연은 가계부채 · 신용보고서 공개"라는 제목을 통해 미국 가계 부채와 신용보고서에 대한 내용을 다룹니다. 0:13에서는 가계 부채 및 신용보고서에 대해 다루고, 1:35에서는 소기업 심리에 대해 다룹니다. 미국 가계 부채와 신용 상황을 파악하고, 소기업들의 경제 상황을 이해하는 데 도움이 되는 정보를 제공합니다. 미국 경제 상황을 파악하고자 하는 투자자들에게 유용한 자료입니다.


이 영상에서는 "미국증시 체크포인트"라는 제목을 통해 미국 주식 시장의 주요 이슈를 다룹니다. 0:08에서는 서비스업 PMI를 체크포인트로 다루고, 1:28에서는 기업 실적을 살펴봅니다. 서비스업 PMI는 미국 경제의 주요 지표 중 하나로, 주식 시장에 큰 영향을 미

In [196]:
youtube['summary'] = summary
youtube.iloc[:,[0,2,3]]

,title,video_url,summary
0,"[슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이? #중국증시 #로보락",https://www.youtube.com/watch?v=JrHgv0R8RIQ&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=1,"이 영상에서는 ""중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이?""라는 제목을 통해 중국 부동산 시장과 가전 업체들의 주가 상승에 대한 연관성을 탐구합니다. 0:25에서는 중국 매크로를 점검하고, 3:35에서는 중국 해외 무역 비중 변화를 살펴봅니다. 5:21에서는 중국 주요 가전 업체들의 주가를 분석합니다. 중국 부동산 시장의 침체에도 불구하고 가전 업체들의 주가는 상승하고 있으며, 이는 중국의 해외 무역 비중 변화와 관련이 있을 수 있습니다. 이 영상은 중국 투자 전략을 고민하는 투자자들에게 유용한 정보를 제공합니다."
1,"[슬기로운 투자 생각] 중국 고배당주 ETF, 투자 열풍 이유는? #Hangseng #중국주식 #고배당주",https://www.youtube.com/watch?v=u1j5cB9Dqpc&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=2,"이 영상에서는 ""중국 고배당주 ETF, 투자 열풍 이유는?""이라는 제목을 통해 중국 고배당주 투자 열풍에 대해 분석합니다. 0:49에서는 중국 배당주 투자 열풍에 대해 다루고, 1:37에서는 투자 대안의 부재와 높은 배당 수익률에 대해 설명합니다. 03:50에서는 중국판 밸류업인 신국9조에 대해 이야기하고, 05:08에서는 홍콩 배당주의 세제 혜택에 대해 분석합니다. 06:35에서는 배당+00이 더욱 중요하다는 것을 강조하며, 09:10에서는 중국과 홍콩 증시의 새로운 투자 물결로서 고배당 투자를 제안합니다. 이 영상은 중국 고배당주 투자에 대한 다양한 측면을 분석하고, 투자자들에게 유용한 정보를 제공합니다. 중국 고배당주 투자에 관심이 있는 투자자들에게 추천합니다."
2,"반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해"" #CATL | 슬기로운 투자 생각",https://www.youtube.com/watch?v=Ms6p1HznC_U&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=3,"이 영상은 ""반등 랠리 기대주 찾기 ""주가 하락, 나는 진짜 억울해""""라는 제목으로, 글로벌 1위 배터리 업체인 CATL을 다룹니다. 영상에서는 CATL이 주가 하락에도 불구하고 진짜 억울한 이유와 한중 주요 배터리 제조업체의 수익성을 비교합니다. 또한, CATL의 주가 및 PER 밴드를 분석하여 투자자들에게 유용한 정보를 제공합니다. 결론적으로, 이 영상은 CATL의 현재 상황과 미래 전망을 분석하여 투자자들에게 반등 랠리 기대주를 찾는 데 도움을 줍니다."
3,"🌝중국 주도주 반등 랠리, 제대로 시동겁니다. 🚌 놓치기 전에 타세요~ [슬기로운 투자 생각] ep.7 #중국주식 #주도주",https://www.youtube.com/watch?v=7Yoej1DUR_0&list=PL1-YXgQy7nWwKzL4VwseN46vEIJ-pxmA6&index=4,"이 영상에서는 ""중국 주도주 반등 랠리, 제대로 시동 겁니다""라는 제목을 통해 중국 주식 시장의 주도주에 대한 분석을 다룹니다. 0:31에서는 최근 주가 하락에 대한 의견을 제시하고, 1:31에서는 글로벌 1위 배터리 업체를 중심으로 반등 랠리에 대한 분석을 제시합니다. 2:18에서는 한중 주요 배터리 제조업체의 수익성을 비교하고, 2:53에서는 CATL의 주가 및 PER 밴드를 분석합니다. 이 영상은 중국 주식 시장의 주도주와 관련된 주요 동향과 분석을 제공합니다. 중국 주식 투자에 관심이 있는 분들에게 유용한 정보를 제공할 것입니다."
4,[유토피아] AI는 Apple iphone의 줄임말? | 2024.5.17(금),https://www.youtube.com/watch?v=GDHMzjCLSjk&list=PL1-YXgQy7nWx-kLXcBKDmf8IdGAdyAbpu&index=1,"이 영상에서는 ""AI는 Apple iPhone의 줄임말?""이라는 제목을 통해 AI와 Apple의 관계를 탐구합니다. 0:08에서 본격적으로 시작하며, 0:33에는 Open AI의 Spring Update에 대해 다룹니다. 이어서 4:40에는 최신 인공지능 모델 GPT-4o의 성능을 분석하고, 6:00에는 AI Peer Table에 대한 내용을 소개합니다. Open AI는 인공지능 연구소로 GPT-4o는 더욱 발전된 자연어 처리 능력을 자랑합니다. 이 영상은 AI 관련 주요 업데이트와 기술의 발전을 이해하는 데 도움을 줍니다. 결론적으로, AI의 최신 동향을 파악하고 싶다면 이 영상을 시청하는 것이 유용합니다."
...,...,...,...
1444,"[글로벌모닝브리핑] 미국 증시, 과도한 금리인하 기대에 대한 경계에 하락ㅣ2024.1.17(수)",https://www.youtube.com/watch?v=HDapc9owarw&list=PL1-YXgQy7nWwV8g0DYsVXR6-TTqm12L1H&index=126,"이 영상에서는 ""미국 증시, 과도한 금리 인하 기대에 대한 경계에 하락""이라는 제목을 통해 미국 증시의 상황을 분석합니다. 최근 미국 증시가 과도한 금리 인하 기대에 대한 경계로 인해 하락하고 있다는 내용을 다룹니다. 글로벌 시장 동향과 특징 종목에 대한 정보를 매일 아침 신속하게 전달드리며, 스마트머니 채널을 구독하시면 실시간 방송 알림을 받아보실 수 있습니다. 미국 증시의 동향을 파악하고 싶은 투자자들에게 유용한 정보를 제공합니다."
1445,"[글로벌모닝브리핑] 유럽 증시, 매파적 발언과 독일 경제 위축에 하락ㅣ2024.1.16(화)",https://www.youtube.com/watch?v=d4v3Oagj9z4&list=PL1-YXgQy7nWwV8g0DYsVXR6-TTqm12L1H&index=127,"이 영상에서는 ""유럽 증시, 매파적 발언과 독일 경제 위축에 하락""이라는 제목을 통해 유럽 증시의 상황을 살펴봅니다. 0:00에서는 오프닝을 시작으로 1:03에는 글로벌 팩트체크를 진행합니다. 6:38에는 마켓워치 PLUS를, 6:54에는 유럽시장 Review를 다룹니다. 7:41에는 오늘의 키워드로 독일 GDP와 매파 발언을 선정하여 분석합니다. 마지막으로 14:40에는 한국 시장에 대한 Preview를 제공합니다. 이 영상은 유럽 증시와 관련된 주요 이슈와 시장 동향을 파악하는 데 도움을 주며, 한국 시장에 대한 예측도 제공합니다."
1446,"[글로벌모닝브리핑] 미국 증시, 생산자물가 감소에도 실적 부진에 혼조세로 끝나ㅣ2024.1.15(월)",https://www.youtube.com/watch?v=OKoJkCgORCY&list=PL1-YXgQy7nWwV8g0DYsVXR6-TTqm12L1H&index=128,"이 영상에서는 ""미국 증시, 생산자물가 감소에도 실적 부진에 혼조세로 끝나""라는 제목을 통해 미국 증시의 상황을 분석합니다. 0:00에서는 오프닝을 시작으로 1:04에서는 글로벌 팩트체크를 진행합니다. 7:46에는 마켓워치 PLUS를, 8:04에는 미국시장 Review를 다룹니다. 오늘의 키워드는 PPI와 어닝시즌이며 17:28에는 한국시장 Preview를 진행합니다. 이 영상은 미국 증시와 관련된 주요 이슈와 시장 동향을 파악하는 데 유용합니다. 미국 주식 투자에 관심이 있다면 이 영상을 시청하는 것이 좋습니다."
1447,"[글로벌모닝브리핑] 미국 증시, 양적긴축 속도 조절 기대감에 혼조세 마감ㅣ2024.1.12(금)",https://www.youtube.com/watch?v=Vs5Fh3RZYSM&list=PL1-YXgQy7nWwV8g0DYsVXR6-TTqm12L1H&index=129,"이 영상에서는 ""미국 증시, 양적긴축 속도 조절 기대감에 혼조세 마감""이라는 제목을 통해 미국 증시의 최근 동향을 살펴봅니다. 1월 12일 금요일의 미국 증시는 양적긴축 속도 조절에

In [109]:
youtube.to_csv("youtube_preprocessing.csv")

### 3. 임베딩

In [13]:
youtube = pd.read_csv("youtube_preprocessing.csv")

youtube = youtube.dropna()
youtube = youtube.replace('', None)
youtube = youtube.dropna()
youtube.reset_index(drop = True, inplace = True)
youtube = youtube.iloc[:, 1:]
len(youtube)

1449

In [14]:
summary = youtube['summary']

In [16]:
# Sentence Transformer 모델 (Hugging Face에서 Most Likes 기준으로 젤 높은 거 가져옴)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 고객 정보 벡터(문장 벡터)
sentence1_embedding = model.encode(customer_feature_text)

# 정제된 유튜브 content의 벡터(문장 벡터)로 결합된 행렬
embeddings = model.encode(summary)

In [17]:
# 각 벡터를 리스트로 감싸서 새로운 리스트 생성
vector_list = [list(vec) for vec in embeddings]

In [24]:
print(len(vector_list))

youtube['embedding'] = vector_list
youtube.head(5)

1449


,title,content,video_url,summary,embedding
0,"[슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨...",【이인구의 슬기로운 투자 생각】 ep4. 중국투자전략\n출연: 이인구 이사(미래에셋...,https://www.youtube.com/watch?v=JrHgv0R8RIQ&li...,"이 영상에서는 ""중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨 일이?...","[0.033734016, 0.04950758, 0.05461615, 0.006250..."
1,"[슬기로운 투자 생각] 중국 고배당주 ETF, 투자 열풍 이유는? #Hangseng...",【이인구의 슬기로운 투자 생각】\nep5. 중국 Go! 배당 주!\n\n00:00 ...,https://www.youtube.com/watch?v=u1j5cB9Dqpc&li...,"이 영상에서는 ""중국 고배당주 ETF, 투자 열풍 이유는?""이라는 제목을 통해 중국...","[0.066883996, 0.007243766, 0.05419471, 0.01784..."
2,"반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해"" #CATL | 슬기로운 투자 생각","【이인구의 슬기로운 투자 생각】\nep.6 반등 랠리 기대주 찾기 ""주가하락, 나는...",https://www.youtube.com/watch?v=Ms6p1HznC_U&li...,"이 영상은 ""반등 랠리 기대주 찾기 ""주가 하락, 나는 진짜 억울해""""라는 제목으로...","[0.061167903, 0.0111971935, 0.07629963, 0.0187..."
3,"🌝중국 주도주 반등 랠리, 제대로 시동겁니다. 🚌 놓치기 전에 타세요~ [슬기로운 ...","【이인구의 슬기로운 투자 생각】\nep.6 반등 랠리 기대주 찾기 ""주가하락, 나는...",https://www.youtube.com/watch?v=7Yoej1DUR_0&li...,"이 영상에서는 ""중국 주도주 반등 랠리, 제대로 시동 겁니다""라는 제목을 통해 중국...","[0.0114629185, 0.051250964, 0.058944806, 0.009..."
4,[유토피아] AI는 Apple iphone의 줄임말? | 2024.5.17(금),#apple #ai #openai\n\n✨ 쏟아지는 AI 관련 홍수 속에서 꼭 알아...,https://www.youtube.com/watch?v=GDHMzjCLSjk&li...,"이 영상에서는 ""AI는 Apple iPhone의 줄임말?""이라는 제목을 통해 AI와...","[0.046036452, -0.0024850473, 0.1074625, -0.026..."


In [21]:
youtube[['title','video_url','summary','embedding']].to_csv("youtube_data.csv")

### 4. 유사도 측정

In [15]:
# 한 문장 벡터를 2차원 배열로 변환
sentence1_embedding_2d = sentence1_embedding.reshape(1, -1)

# 코사인 유사도 계산
similarity = cosine_similarity(sentence1_embedding_2d, embeddings)
print(f"Cosine Similarity:\n{similarity}")


Cosine Similarity:
[[0.9271742  0.8783695  0.7657192  ... 0.90339625 0.86872935 0.8600447 ]]


### 5. 유사도 순으로 내림차순 정렬

In [132]:
pd.set_option("display.max_rows", 10)

In [192]:
# df = pd.DataFrame(columns = ['title', 'content', 'similarity', 'url'])
# df['title'] = youtube.title[:30]
# df['content'] = summary[:30]
# df['similarity'] = similarity[0][:30]
# df['url'] = youtube.video_url[:30]
# df.sort_values(by = 'similarity', ascending = False)

df = pd.DataFrame(columns = ['title', 'content', 'similarity', 'url'])
df['title'] = youtube.title
df['content'] = summary
df['similarity'] = similarity[0]
df['url'] = youtube.video_url
df = df.sort_values(by = 'similarity', ascending = False).iloc[:30]
df

,title,content,similarity,url
444,[🙋마이턴] EP.3 내 자식의 성공은 곧 나의 성공! VS내 노후는 아무도 책임져주지 않아! l 2022.11.09(수),"이 영상에서는 ""내 자식의 성공은 곧 나의 성공! VS 내 노후는 아무도 책임져주지 않아!""라는 주제로 자녀 주식 투자와 나의 노후 준비에 대해 토론합니다. 나루맘 박수진 연구위원과 2남2녀 김석환 선임연구위원이 서로 다른 스타일로 치열한 토론을 펼치며, 시청자들은 자신이 공감하는 의견에 투표하고 응원 댓글을 작성할 수 있습니다. 이를 통해 금융과 경제에 대한 다각적인 시각을 얻을 수 있으며, 자신의 노후 준비와 자녀 주식 투자에 대한 고민을 함께 나눌 수 있습니다.",0.945077,https://www.youtube.com/watch?v=OCi3PgmBsJw&list=PL1-YXgQy7nWxtpKbHMPjhD_nSNcWE5d8U&index=3
797,[박수진의 보이는 차이나] 불안함이 더 커진 중국 증시 I 2024.1.22(월),"이 영상에서는 ""불안함이 더 커진 중국 증시""라는 제목을 통해 중국 증시에 대한 불안감을 분석합니다. 0:04에서 오프닝 멘트를 시작으로, 0:09에서는 이번 주 중화권 주요 경제 일정을 다룹니다. 1:53에서는 아직 불안과 불신이 더 큰 시장이라는 주제로 현재 중국 시장의 분위기를 전달합니다. 2:01에서는 2024년 1월 효과가 없었다는 점을 지적하며 중국 증시의 상황을 분석합니다. 전반적으로 이 영상은 중국 증시의 현재 상황과 주요 경제 일정을 다루며, 중국 시장에 대한 이해를 높이는 데 도움을 줍니다. 중국 시장에 관심이 있는 투자자들에게 유용한 정보를 제공합니다.",0.945047,https://www.youtube.com/watch?v=lievcnLK_Gg&list=PL1-YXgQy7nWxwN_Z96xT52iUWzOq09Wb0&index=10
850,[박수진의 보이는 차이나] 우리는 아직 목마르다! #다음주 확인이 필요한 것들 I 2022.10.20(목),"이 영상에서는 ""우리는 아직 목마르다! 다음주 확인이 필요한 것들""이라는 제목을 통해 중국 증시와 관련된 주요 이슈들을 다룹니다. 0:06부터 중국 증시가 조금씩 상승을 시도하고 있지만 아직은 부족하다는 내용으로 시작합니다. 0:35에는 중국 증시 반등의 조건에 대해 논의합니다. 01:38에는 제로코로나 정책에 대한 중국의 고집을 다루고, 02:53에는 거래대금 7~8천억 위안대 안착했다는 내용을 다룹니다. 03:58에는 전기차, 태양광 등 대표 기업들의 3분기 호실적 예고에 대해 다루고 05:55에는 다음주에 꼭 확인해야 할 필수 사항들을 정리합니다. 07:40에는 10월 16일 시진핑 주석의 업무 보고 주요 내용을 요약하고, 09:46에는 오늘의 결론을 제시합니다. 이 영상은 중국 증시와 관련된 다양한 이슈들을 다루고 있어, 중국 경제에 관심이 있는 분들에게 유용한 정보를 제공합니다.",0.942194,https://www.youtube.com/watch?v=PJvv4gIGyB0&list=PL1-YXgQy7nWxwN_Z96xT52iUWzOq09Wb0&index=63
758,"[유명간의 어닝스리비전] ""내년 실적 하향 조정도 불가피""ㅣ2022.09.26 (월)","이 영상에서는 ""내년 실적 하향 조정도 불가피""라는 제목을 통해 2023년 기업 실적 전망에 대해 논의합니다. 유명간의 어닝스 리비전(주간 이익동향) 시리즈로, 전 세계 주식시장의 이익 동향을 정리합니다. 매주 월요일 오전에 새로운 에피소드가 공개됩니다. 9월 5주차 어닝스 리비전 보고서를 통해 자세한 내용을 확인할 수 있습니다. 미래에셋 스마트머니 채널을 구독하여 실시간 방송 알림을 받아볼 수도 있습니다.",0.937939,https://www.youtube.com/watch?v=AnP-ZLcNXFA&list=PL1-YXgQy7nWwV744omocDORZQP02LsRfK&index=61
818,[박수진의 보이는 차이나] 나쁜 소식&좋은 소식I 2023.07.17(월),"이 영상에서는 ""나쁜 소식&좋은 소식I""이라는 제목으로 중국 경제와 관련된 다양한 주제를 다룹니다. 0:00에서는 오프닝 멘트를 시작으로, 0:18에서는 이번 주 중화권 증시 주요 일정을 소개합니다. 1:50에서는 6월 경제 성적표를 분석하고, 3:40에서는 빅테크 규제가 마무리되고 있다는 소식을 전합니다. 8:50에서는 플랫폼 기업 규제 완화 시그널을 다루고, 9:53에서는 향후 중국 증시의 관전 포인트를 제시합니다. 이 영상은 중국 경제와 관련된 주요 이슈와 트렌드를 이해하는 데 도움을 주며, 중국 증시에 관심 있는 투자자들에게 유용한 정보를 제공합니다.",0.937376,https://www.youtube.com/watch?v=GSK8OqQXaBc&list=PL1-YXgQy7nWxwN_Z96xT52iUWzOq09Wb0&index=31
...,...,...,...,...
847,[박수진의 보이는 차이나] 위드코로나로 가는 길 #중국 증시 Q&A I 2022.11.17(목),"이 영상에서는 ""위드코로나로 가는 길""이라는 제목을 통해 중국의 코로나 대응 정책 변화와 그에 따른 경제적 영향을 살펴봅니다. 0:06부터 시작하여, 중국이 어떻게 위드코로나로 전환하고 있는지에 대한 내용을 다룹니다. 05:49부터는 시청자들의 중국 증시에 대한 질문에 답합니다. 첫 번째 질문은 시진핑 3기 정권 발표 이후 외국인 자금의 차이나런에 대한 내용이고, 두 번째 질문은 중국 제약바이오 산업의 최근 성장세에 대한 내용입니다. 14:00에는 오늘의 결론을 제시하며, 중국의 정책 변화와 그에 따른 산업 및 경제적 영향을 분석합니다. 이 영상은 중국의 코로나 대응 정책 변화와 그에 따른 경제적 영향에 관심이 있는 투자자들에게 유용한 정보를 제공합니다.",0.931954,https://www.youtube.com/watch?v=CF2uTM78Y3Q&list=PL1-YXgQy7nWxwN_Z96xT52iUWzOq09Wb0&index=60
953,[연금 포트폴리오 서비스] 개인연금 랩(Wrap),"이 영상에서는 ""개인연금 랩""이라는 제목을 통해 개인연금 포트폴리오 서비스에 대해 설명합니다. 바쁜 고객, 소액 투자자들도 수수료 걱정 없이 연금을 관리할 수 있는 서비스입니다. 영상에서는 개인연금 랩의 개념, 4가지 유형, 특징, 가입 방법에 대해 다룹니다. 또한, 개인연금 랩 관련 이벤트도 안내하고 있으니, 관심 있는 분들은 이벤트 상세페이지를 확인해 보시기 바랍니다. 이 영상은 개인연금 포트폴리오를 효율적으로 관리하고자 하는 분들에게 유용한 정보를 제공합니다.",0.931785,https://www.youtube.com/watch?v=o9p9krKoiPk&list=PL1-YXgQy7nWzD07InBb0znPABUexTyJEp&index=2
997,[언박싱연금📦](4K) 연금으로 부동산 투자가 가능하다고?! 어머!,"이 영상에서는 ""연금으로 부동산 투자가 가능하다고?!""라는 제목으로 연금 계좌를 통한 부동산 투자 방법을 다룹니다. 증권업계 최초로 미래에셋증권 개인연금계좌에서 리츠 매매 서비스가 시행된다는 소식을 전하며, 적은 금액으로 쉽게 부동산에 투자할 수 있는 리츠의 특징과 선택 시 주의점을 알려줍니다. 또한, 리츠 관련 정보를 확인할 수 있는 리츠정보시스템과 금융감독원 전자공시시스템의 링크를 제공합니다. 연금을 통한 부동산 투자에 관심이 있다면 이 영상을 시청하는 것이 도움이 될 것입니다.",0.931596,https://www.youtube.com/watch?v=e20zRRNy5Cs&list=PL1-YXgQy7nWzD07InBb0znPABUexTyJEp&index=46
1102,[리서치 포럼 2022] 그린플레이션의 역설,"이 영상에서는 ""그린플레이션의 역설""이라는 제목을 통해 그린 에너지 시장의 현재 상황과 미래 전망을 다룹니다. 0:28부터 시작하여, 올해 그린 에너지 주가가 부진한 이유를 분석하고, 1:39에서는 '그린플레이션'의 개념에 대해 설명합니다. 4:33에서는 '그린플레이션'이 시장 위축으로 이

### 6. Json 파일로 저장

In [193]:
json_path = "youtube_recommendation.json"
df.to_json(json_path, orient="records", lines=True, force_ascii=False)

In [7]:
json_path = "youtube_recommendation.json"
df_loaded = pd.read_json(json_path, orient="records", lines=True)
df_loaded

,title,content,similarity,url
0,[🙋마이턴] EP.3 내 자식의 성공은 곧 나의 성공! VS내 노후는 아무도 책임져...,"이 영상에서는 ""내 자식의 성공은 곧 나의 성공! VS 내 노후는 아무도 책임져주지...",0.945077,https://www.youtube.com/watch?v=OCi3PgmBsJw&li...
1,[박수진의 보이는 차이나] 불안함이 더 커진 중국 증시 I 2024.1.22(월),"이 영상에서는 ""불안함이 더 커진 중국 증시""라는 제목을 통해 중국 증시에 대한 불...",0.945047,https://www.youtube.com/watch?v=lievcnLK_Gg&li...
2,[박수진의 보이는 차이나] 우리는 아직 목마르다! #다음주 확인이 필요한 것들 I ...,"이 영상에서는 ""우리는 아직 목마르다! 다음주 확인이 필요한 것들""이라는 제목을 통...",0.942194,https://www.youtube.com/watch?v=PJvv4gIGyB0&li...
3,"[유명간의 어닝스리비전] ""내년 실적 하향 조정도 불가피""ㅣ2022.09.26 (월)","이 영상에서는 ""내년 실적 하향 조정도 불가피""라는 제목을 통해 2023년 기업 실...",0.937939,https://www.youtube.com/watch?v=AnP-ZLcNXFA&li...
4,[박수진의 보이는 차이나] 나쁜 소식&좋은 소식I 2023.07.17(월),"이 영상에서는 ""나쁜 소식&좋은 소식I""이라는 제목으로 중국 경제와 관련된 다양한 ...",0.937376,https://www.youtube.com/watch?v=GSK8OqQXaBc&li...
5,[2024 리서치 전망] 지속가능투자 / 에너지 - 신재생 에너지 산업의 돌파구는?,"이 영상에서는 ""지속가능투자 / 에너지 - 신재생 에너지 산업의 돌파구는?""이라는 ...",0.937029,https://www.youtube.com/watch?v=ZSuLH79dlgA&li...
6,"[주식食객: 기업 맛집을 찾아서] Ep.10 - 저평가 받는 국내 은행주, 주주환원...","이 영상에서는 ""저평가 받는 국내 은행주, 주주환원정책 확대에 주목""이라는 제목으로...",0.936597,https://www.youtube.com/watch?v=SL7p3r_nrGA&li...
7,푸른씨앗이 선생님들의 노후준비를 응원합니다!,"이 영상은 ""푸른씨앗이 선생님들의 노후준비를 응원합니다!""라는 제목으로, 어린이집 ...",0.936072,https://www.youtube.com/watch?v=e6-5jqRZS-Q&li...
8,[박수진의 보이는 차이나] 중요한 건 꺾이지 않는 마음 I 2022.12.08(목),"이 영상에서는 ""중요한 건 꺾이지 않는 마음""이라는 제목으로 중국 증시와 관련된 경...",0.936046,https://www.youtube.com/watch?v=-kDBNPsXl1I&li...
9,[Global Talk] 인도네시아 투자 가이드,"이 영상에서는 ""인도네시아 투자 가이드""라는 제목으로 인도네시아 투자에 대한 정보를...",0.935916,https://www.youtube.com/watch?v=d93Ulbs3IZM&li...


# ========================================

ChromaDB 실험

In [16]:
youtube['embedding']

0       [0.033734016, 0.04950758, 0.05461615, 0.006250...
1       [0.066883996, 0.007243766, 0.05419471, 0.01784...
2       [0.061167903, 0.0111971935, 0.07629963, 0.0187...
3       [0.0114629185, 0.051250964, 0.058944806, 0.009...
4       [0.046036452, -0.0024850473, 0.1074625, -0.026...
                              ...                        
1444    [0.03241703, 0.024514697, 0.08182656, 0.028243...
1445    [0.011647342, 0.0027475504, 0.050104458, 0.033...
1446    [0.032828085, 0.017825348, 0.083292425, 0.0545...
1447    [0.053962108, 0.021007635, 0.06093197, -0.0025...
1448    [0.0017246775, 0.040114097, 0.062765285, 3.873...
Name: embedding, Length: 1449, dtype: object

In [6]:
import chromadb
from chromadb.config import Settings
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

client = chromadb.PersistentClient(path="my_chroma_db")
# create a collection
collection = client.get_collection(name = "youtube_embedding")


In [71]:
ids = []
embeddings = []
metadatas = []

for idx, row in youtube.iterrows():
    ids.append(str(idx))
    embeddings.append([float(x) for x in row['embedding']])
    metadatas.append({
        'title' : row['title'],
        'url' : row['video_url']
    })

# 저장 완료
collection.upsert(
    ids = ids,
    embeddings = embeddings,
    metadatas = metadatas
)

저장된 Chromadb 다시 불러오기

In [7]:
# ChromaDB 클라이언트 초기화
client = chromadb.PersistentClient(path="my_chroma_db")

# collection 가져오기
collection = client.get_collection(name="youtube_embedding")

In [12]:
collection.query()

ValueError: You must provide one of query_embeddings, query_texts, query_images, or query_uris.

In [3]:
# 코사인 유사도 기반으로 검색
def cosine_similarity_search(query_vector, embeddings, metadatas, top_n):
    # 코사인 유사도 계산
    similarities = cosine_similarity([query_vector], embeddings)[0]
    # 유사도에 따라 상위 N개 선택
    top_indices = similarities.argsort()[-top_n:][::-1]
    
    # 결과 출력
    results = []
    for idx in top_indices:
        result = {
            "title": metadatas[idx]['title'],
            "url": metadatas[idx]['url'],
            "embedding": embeddings[idx],
            "similarity": similarities[idx]
        }
        results.append(result)
    
    return results

# 저장된 임베딩 벡터 가져오기
stored_embeddings = np.array([embedding for embedding in embeddings])
stored_metadatas = metadatas

# 코사인 유사도 기반 검색 결과
query_vector = sentence1_embedding_2d.tolist()[0]
embeddings = stored_embeddings
metadatas = stored_metadatas
top_n = 5
results = cosine_similarity_search(query_vector, embeddings, metadatas, top_n)

# 검색된 데이터 출력
for result in results:
    print(f"Title: {result['title']}, URL: {result['url']}, Similarity: {result['similarity']}")

NameError: name 'embeddings' is not defined

In [79]:
import json

# 검색된 데이터 출력 및 JSON 파일로 저장
output_data = []
for result in results:
    print(f"Title: {result['title']}, URL: {result['url']}, Similarity: {result['similarity']}")
    output_data.append({
        "title": result['title'],
        "url": result['url'],
        "similarity": result['similarity']
    })

# JSON 파일로 저장
with open("youtube_recommendation.json", "w", encoding='utf-8') as json_file:
    json.dump(output_data, json_file, ensure_ascii=False, indent=4)

Title: [🙋마이턴] EP.3 내 자식의 성공은 곧 나의 성공! VS내 노후는 아무도 책임져주지 않아! l 2022.11.09(수), URL: https://www.youtube.com/watch?v=OCi3PgmBsJw&list=PL1-YXgQy7nWxtpKbHMPjhD_nSNcWE5d8U&index=3, Similarity: 0.9450771397746252
Title: [박수진의 보이는 차이나] 불안함이 더 커진 중국 증시 I 2024.1.22(월), URL: https://www.youtube.com/watch?v=lievcnLK_Gg&list=PL1-YXgQy7nWxwN_Z96xT52iUWzOq09Wb0&index=10, Similarity: 0.9450471090138646
Title: [박수진의 보이는 차이나] 우리는 아직 목마르다! #다음주 확인이 필요한 것들 I 2022.10.20(목), URL: https://www.youtube.com/watch?v=PJvv4gIGyB0&list=PL1-YXgQy7nWxwN_Z96xT52iUWzOq09Wb0&index=63, Similarity: 0.9421933933446127
Title: [유명간의 어닝스리비전] "내년 실적 하향 조정도 불가피"ㅣ2022.09.26 (월), URL: https://www.youtube.com/watch?v=AnP-ZLcNXFA&list=PL1-YXgQy7nWwV744omocDORZQP02LsRfK&index=61, Similarity: 0.9379387044883749
Title: [박수진의 보이는 차이나] 나쁜 소식&좋은 소식I 2023.07.17(월), URL: https://www.youtube.com/watch?v=GSK8OqQXaBc&list=PL1-YXgQy7nWxwN_Z96xT52iUWzOq09Wb0&index=31, Similarity: 0.9373758074987435


In [77]:
# JSON 파일로부터 데이터 불러오기
with open("youtube_recommendation.json", "r") as json_file:
    loaded_data = json.load(json_file)

# 불러온 데이터 출력
for result in loaded_data:
    print(f"Title: {result['title']}, URL: {result['url']}, Similarity: {result['similarity']}")


Title: [🙋마이턴] EP.3 내 자식의 성공은 곧 나의 성공! VS내 노후는 아무도 책임져주지 않아! l 2022.11.09(수), URL: https://www.youtube.com/watch?v=OCi3PgmBsJw&list=PL1-YXgQy7nWxtpKbHMPjhD_nSNcWE5d8U&index=3, Similarity: 0.9450771397746252
Title: [박수진의 보이는 차이나] 불안함이 더 커진 중국 증시 I 2024.1.22(월), URL: https://www.youtube.com/watch?v=lievcnLK_Gg&list=PL1-YXgQy7nWxwN_Z96xT52iUWzOq09Wb0&index=10, Similarity: 0.9450471090138646
Title: [박수진의 보이는 차이나] 우리는 아직 목마르다! #다음주 확인이 필요한 것들 I 2022.10.20(목), URL: https://www.youtube.com/watch?v=PJvv4gIGyB0&list=PL1-YXgQy7nWxwN_Z96xT52iUWzOq09Wb0&index=63, Similarity: 0.9421933933446127
Title: [유명간의 어닝스리비전] "내년 실적 하향 조정도 불가피"ㅣ2022.09.26 (월), URL: https://www.youtube.com/watch?v=AnP-ZLcNXFA&list=PL1-YXgQy7nWwV744omocDORZQP02LsRfK&index=61, Similarity: 0.9379387044883749
Title: [박수진의 보이는 차이나] 나쁜 소식&좋은 소식I 2023.07.17(월), URL: https://www.youtube.com/watch?v=GSK8OqQXaBc&list=PL1-YXgQy7nWxwN_Z96xT52iUWzOq09Wb0&index=31, Similarity: 0.9373758074987435
